In [1]:
#TODO: clean up old methods of image processing, transfer neural network prep method into a .py script
import os

import matplotlib.pyplot as plt
import matplotlib.mlab as mlb
import numpy as np
import tensorflow as tf
from PIL import Image, ImageOps
import pandas as pd

from sklearn import preprocessing
from skimage import io, filters, morphology, util


from utils.general_utils import get_imgMatrix_from_id, get_random_id, get_id_from_filename, get_breed_value_from_id, populate_breeds, get_label_array_from_id, get_random_id



In [2]:
#parameters to change
np.set_printoptions(threshold=np.nan) #numpy arrays will print the entire array now

#create a .csv file of ids and image_matricies and save it to the data/preprocessed folder


#current directory to process
dir="../data/included/Test/"

BREED_LIST = "../data/preprocessed_data/breed_list.csv"

#prepare the breed list dataframe
labels = populate_breeds(BREED_LIST) #get the list of all dog breeds
labels_np = np.array(labels).reshape(120,1) #labels list reshaped to numpy array

In [ ]:
def getImageDimensions(file_name, original_dir="../data/included/Train/"):
    file = original_dir + "" + file_name #store the file name and location  
    image = Image.open(file)
    width, height = image.size
    return width, height

def dimensionAnalysis(dir="../data/included/Test/"):
    data_files = os.listdir(dir) #get a list of all filenames from Train dir
    
    n = 0
    x_total = 0
    y_total = 0
    x_max = 0
    y_max = 0
    
    for file in data_files:
        x = 0
        y = 0
        x,y = getImageDimensions(file)
        n+=1
        x_total+= x
        y_total+= y        
        
        if(x>x_max):
            x_max = x
            print(file)
            print(x)
        if(y>y_max):
            y_max = y
        
    x_avg = x_total/n
    y_avg = y_total/n
    
    print("The avg x dimension is: " + str(x_avg))
    print("The avg y dimension is: " + str(y_avg))
    print("The Maximum width is: " + str(x_max))
    print("The Maximum height is: " + str(y_max))
    
dimensionAnalysis(dir="../data/included/Train")
#solution:
#  if width is under 500 add padding to make the image 500 x 500
#  if height is under 500 add padding to make the image 500 x 500
#  if either dimension > 500 scale down to get one dimension == to 500
#     then add padding to get the other dimension == to 500

In [ ]:
#convert all images to grayscale vectors-- no longer used as NN will be using 500 x 500 x 3 images as input
def normalize_image(file_name, original_dir="../data/included/Train/",output_dir="/data/preprocessed_data/Train/"):
    file = original_dir + "" + file_name #store the file name and location  
    image = Image.open(file)
    
    #scale the image to 500 x 500 x 3
    
    
    #  if width is under 500 add padding to make the image 500 x 500
    #  if height is under 500 add padding to make the image 500 x 500
    #  if either dimension > 500 scale down to get one dimension == to 500
    #     then add padding to get the other dimension == to 500
    
    width, height = image.size
    
    if(width > 500 or height > 500): #if width is greater than 500 or the height is greater than 500
        if(width > height): #determine which side length is greater
            image_ratio = 500/width #determine ratio of old image to new image
        else:
            image_ratio = 500/height
             
        width = int(width * image_ratio) #calculate new width
        height = int(height * image_ratio) #calculate new height
        image = image.resize((500, height), resample=1) #scale image to 
    
        width, height = image.size #refresh the width and height to make sure everything is still accurate
        delta_h = 500 - height #calculate the amount of height padding
        delta_w = 500 - width #calculate the amount of width padding -- should be 0
        padding = (delta_w//2, delta_h//2, delta_w-(delta_w//2), delta_h-(delta_h//2))
        image = ImageOps.expand(image, padding)
    
        #print("test1... " + file_name) 
    
    
    width, height = image.size
    #after neither size is greater than 500 or already scaled
    if(width < 500 or height < 500): #if either dimension is less than 500 padding is added
        #add padding to the height and width to change the image dimensions to 500 x 500 x 3
        width, height = image.size #refresh the width and height to make sure everything is still accurate
        delta_h = 500 - height #calculate the amount of height padding
        delta_w = 500 - width #calculate the amount of width padding -- should be 0
        padding = (delta_w//2, delta_h//2, delta_w-(delta_w//2), delta_h-(delta_h//2))
        image = ImageOps.expand(image, padding)
    
    
    image_id = get_id_from_filename(file_name)
    
    width, height = image.size
    if(width != 500 or height != 500):
        print("something went wrong")
    
    #save the image to the output_dir with the same id
    image.save(("/home/ben/Documents/github/kaggle_dog_breed/data/preprocessed_data/Train/"+ "" + image_id + ".png"),format='PNG')    
        
        
    return (file_name + " ... saved")


#type(plt.imread("../data/included/Train/bd6a14ec09f86f3fef46f9433db8c445.jpg"))
#plt.imread("../data/included/Train/" + image)


In [ ]:
#pre-process all of the Train files
def pre_process(dir="../data/included/Train/"):
    data_files = os.listdir(dir) #get a list of all filenames from Train dir
    
    for file in data_files:
        normalize_image(file)
    
    
    return(True)
pre_process()

In [ ]:
#load random image from training data
img_loc = "../data/preprocessed_data/Train/" + get_random_id(dir="../data/included/Train/") + ".png"
img = io.imread(img_loc, as_grey=False) #import image in greyscale

#display the image
io.imshow(img)
io.show()

img.shape

In [4]:
#convert a directory of pre-processed images into a pandas dataframe for easy use with tensorflow
def dataFrameBuilder(dir="../data/preprocessed_data/Train/"):
    df = pd.DataFrame(columns=['ID', 'Image Data', 'Breed'])
    d = []
    
    data_files = os.listdir(dir) #get a list of all filenames from Train dir
    counter = 0
    segmenter = len(data_files)
    
    for file in data_files:
        
        file_id = get_id_from_filename(file)
        data = get_imgMatrix_from_id(file_id)
        breed_matrix = get_label_array_from_id(file_id, labels_np)
        
        d.append({'ID': file_id, 'Image Data': data, 'Breed': breed_matrix})
       
    
        counter+=1
        
        if(counter > 10): #every 10 indexes in order to preserve ram
            df_temp = pd.DataFrame(d, columns=['ID', 'Image Data', 'Breed']) #store list in a temp dataframe            
            
            df = pd.concat([df, df_temp]) #concatenate the temp df onto the end of df
            
            d = [] #clear the list 
            counter = 0 #restart the counter
    
    df_temp = pd.DataFrame(d) #initialize the DataFrame
    df = pd.concat([df, df_temp])
    df_temp = None
    
    return df
    
df = dataFrameBuilder()

df.sample(10)

KeyboardInterrupt: 

In [ ]:
df.to_csv("/home/ben/Documents/github/kaggle_dog_breed/data/preprocessed_data/train.csv", del='\')

In [ ]:
file_id = get_random_id()

#file_id = get_id_from_filename(file)
data = get_imgMatrix_from_id(file_id)
breed_matrix = get_label_array_from_id(file_id, labels_np)



print(data)

In [ ]:
plt.pie([square_cnt, long_horz_cnt, long_vert_cnt], labels=["square", "long horizontal", "long vertical"])

plt.show() 

list_of_widths = sorted(list_of_widths)
list_of_heights = sorted(list_of_heights)

#determine distribution of heights and widths of the images
plt.plot(list_of_widths, mlb.normpdf(list_of_widths, np.mean(list_of_widths), np.std(list_of_widths)), '-o')
plt.hist(list_of_widths, normed=True)
plt.show()

plt.plot(list_of_heights, mlb.normpdf(list_of_heights, np.mean(list_of_heights), np.std(list_of_heights)), '-o')
plt.hist(list_of_heights, normed=True)
plt.show()

print("Minimum image width: ", np.min(list_of_widths))
print("Minimum image height: ", np.min(list_of_heights))